## Reacher with obstacle avoidance

This is the code to train the reaching armature affordance model with an overhead camera as sensor input. For more detail about the motivation behind network architecture choices and the training procedure, check out the  Hexapod notebook in the other directory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import glob
import sys

from math import *
import pickle

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim
from torch.autograd import Variable

from multiprocessing import Pool

import pybullet as p

import copy

def tovar(x):
    return Variable(torch.FloatTensor(x).cuda(), requires_grad = False)

def toivar(x):
    return Variable(torch.LongTensor(x).cuda(), requires_grad = False)

### Simulation

This class contains code to instantiate and simulate the reaching armature and its sensor inputs in pybullet. We'll mostly use this to generate a large static dataset for training, and for testing the final trained model. 

In [ ]:
class Simulation():
    def __init__(self, angles, objs = [], gui=False):
        if gui:
            self.pid = p.connect(p.GUI)
        else:
            self.pid = p.connect(p.DIRECT)
        
        pid = self.pid        
        self.view = p.computeViewMatrix([0,0,25], [0,0,0], [0,1,0])
        self.proj = p.computeProjectionMatrixFOV(60, 1.0, 0.5, 25.0)
        
        planeId = p.loadURDF("plane.urdf", physicsClientId=pid)
        p.createConstraint(planeId,-1,-1,-1,p.JOINT_FIXED,\
                           [0,0,1],[0,0,0],[0,0,0],physicsClientId=pid)
        
        for i in range(len(objs)):
            obj = p.loadURDF(objs[i][0], objs[i][1], physicsClientId=pid)
            p.createConstraint(obj,-1,-1,-1,p.JOINT_FIXED,\
                              [0,0,1],[0,0,0],objs[i][1],physicsClientId=pid)
            
        cubeStartPos = [0,0,0]
        cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
        self.boxId = p.loadURDF("arm.urdf", cubeStartPos, cubeStartOrientation, physicsClientId=pid)
        p.createConstraint(self.boxId,-1,-1,-1,p.JOINT_FIXED,\
                           [0,0,1],[0,0,0],[0,0,0],physicsClientId=pid)
    
        links = np.arange(8)
        forces = [8000 for x in np.arange(8)]
        vels = [0 for x in np.arange(8)]
        pgain = [0.2 for x in np.arange(8)]
        vgain = [0.6 for x in np.arange(8)]
        p.setJointMotorControlArray(self.boxId, links, p.POSITION_CONTROL, \
                                    targetPositions = angles, \
                                    forces = forces, \
                                    targetVelocities = vels, \
                                    positionGains = pgain, \
                                    velocityGains = vgain, \
                                    physicsClientId=pid)

        for i in range(10):
            p.stepSimulation(physicsClientId=pid)

    def getState(self):
        state = []
        boxId = self.boxId
        pid = self.pid
        
        for i in range(8):
            lstate = p.getLinkState(boxId,i,physicsClientId=pid)
            state.append(lstate[0][0])
            state.append(lstate[0][1])
            state.append(lstate[0][2])
            
        return np.array(state)
    
    def getInState(self):
        camera = p.getCameraImage(width=24, height=24, \
                                  viewMatrix=self.view, projectionMatrix=self.proj, \
                                  renderer = p.ER_TINY_RENDERER, physicsClientId=self.pid)
        
        state = np.array(camera[3])
        return state.ravel()

    def simulateActions(self,acts,delay=0):
        boxId = self.boxId
        pid = self.pid
        links = np.arange(8)
        forces = [8000 for x in np.arange(8)]
        vels = [0 for x in np.arange(8)]
        pgain = [0.2 for x in np.arange(8)]
        vgain = [0.6 for x in np.arange(8)]
        
        states = []
                
        for j in range(200):
            if delay:
                time.sleep(delay)

            p.setJointMotorControlArray(boxId, links, p.POSITION_CONTROL, \
                                        targetPositions = acts, \
                                        forces=forces, \
                                        targetVelocities = vels, \
                                        positionGains = pgain, \
                                        velocityGains = vgain, \
                                        physicsClientId=pid)

            p.stepSimulation(physicsClientId=pid)
        
        states = self.getState()
    
        return np.array(states)
    
    def stop(self):
        p.disconnect(self.pid)

In [ ]:
# Code for generating environments and training data

def simulate(x):
    s = Simulation(np.zeros(8),objs=x[1])
    a = s.simulateActions(x[0])
    s.stop()
    return a

def getSimsStates(BS=10):
    olist = []
    states = []
    for i in range(BS):
        nobj = np.random.randint(6)
        objlist = []
        for j in range(nobj):            
            if np.random.randint(2)==0:
                otype="cube.urdf"
            else:
                otype="sphere.urdf"
            
            pos = [np.random.rand()*10-5, np.random.rand()*10-5, np.random.rand()*5+2]
            
            while (pos[0]**2+pos[1]**2 < 4.5*4.5):
                pos = [np.random.rand()*10-5, np.random.rand()*10-5, np.random.rand()*5+2]
                
            objlist.append([otype, pos])
        olist.append(objlist)
        s = Simulation(np.zeros(8),objs=objlist)
        states.append(s.getInState())
        s.stop()
    
    states = np.array(states)
    
    return states, olist

def getData(N):
    MESH = 7
    PROPOSALS = MESH*MESH
    states, olist = getSimsStates(BS=N)
    acts = []
    ends = []
    
    a,p = propose.generate(tovar(states), predict)    
    a = a.cpu().data.numpy()
    
    for i in range(N):
        acts.append(np.clip(1.0*(np.random.rand(ACTUATORS)*2-1),-1,1))
    ends = []
    
    pool = Pool(4)
    ends = pool.map(simulate, zip(acts,olist))
    pool.close()
    pool.join()
    
    ends = np.array(ends)
    
    return states, np.array(acts), ends

def getProposeData(N):
    MESH = 7
    PROPOSALS = MESH*MESH
    states, olist = getSimsStates(BS=N)
    acts = []
    ends = []
    
    a,p = propose.generate(tovar(states), predict)    
    a = a.cpu().data.numpy()
    
    for i in range(N):
        acts.append(np.clip(a[i,:,i%PROPOSALS] + 0.1*(np.random.rand(ACTUATORS)*2-1),-1,1))
    ends = []
    
    pool = Pool(4)
    ends = pool.map(simulate, zip(acts,olist))
    pool.close()
    pool.join()
    
    ends = np.array(ends)
    
    return states, np.array(acts), ends

### Predictor network

We first process the sensor inputs through three dense layers, and then merge in the actuator values. This lets the proposal network take advantage of the visual system learned by the predictor to understand the environment, which helps the proposer learn to make use of camera data quickly.

We use a dense architecture for the camera here because it's low enough resolution to permit it, but it would be straightforward to switch to a convolutional network to handle higher resolutions. Note that the Conv1d layers here aren't actually convolutions but are rather just a shorthand to make it easy to processing multiple sets of proposed actions in parallel later on.

In [ ]:
SENSORS = 24*24
OSENSORS = 3*8
HIDDEN = 512
HIDDEN2 = 128
ACTUATORS = 8
MESH = 7

# Measures prediction error
def wDist(x,y):
    return (x-y)**2

# Measures outcome distance
# 7*3 is the offset associated with the segment at the end of the arm,
# so the outcome distance only cares about tip position
def wDist4(x,y):
    return (x[:,7*3:7*3+3,:,:]-y[:,7*3:7*3+3,:,:])**2

class Predict(nn.Module):
    def __init__(self):
        super(Predict,self).__init__()
        
        # 
        self.p1 = nn.Conv1d(SENSORS, HIDDEN, 1).cuda()
        self.p2 = nn.Conv1d(HIDDEN, HIDDEN, 1).cuda()
        self.p3 = nn.Conv1d(HIDDEN, HIDDEN, 1).cuda()
        self.p4 = nn.Conv1d(HIDDEN+ACTUATORS, HIDDEN, 1).cuda()
        self.p5 = nn.Conv1d(HIDDEN, HIDDEN, 1).cuda()
        self.p6 = nn.Conv1d(HIDDEN, HIDDEN, 1).cuda()
        self.p7 = nn.Conv1d(HIDDEN, HIDDEN, 1).cuda()
        self.p8 = nn.Conv1d(HIDDEN, OSENSORS, 1).cuda()
        self.drop = nn.Dropout(p=0.1)
        
        self.adam = torch.optim.Adam(self.parameters(), lr = 2.5e-4)
    
    def process(self,state):
        pz = F.elu(self.p1(1.0-state))
        
        return pz
    
    def predict_hid(self,z,a):
        pz = z + F.elu(self.p3(F.elu(self.p2(z))))
        pz = torch.cat([pz,a],1)
        pz = F.elu(self.p5(F.elu(self.p4(pz))))
        pz = pz + F.elu(self.p7(F.elu(self.p6(pz))))
        
        pzpred = self.p8(pz)
        
        return pzpred
    
    # The depth camera tends to have values near 1.0, with objects being lower,
    # so 1-x is a convenient transform to normalize this a bit
    def predict(self,state,a):
        pz = self.drop(F.elu(self.p1(1.0-state)))
        pz = pz + self.drop(F.elu(self.p3(self.drop(F.elu(self.p2(pz))))))
        pz = torch.cat([pz,a],1)
        pz = self.drop(F.elu(self.p5(self.drop(F.elu(self.p4(pz))))))
        pz = pz + self.drop(F.elu(self.p7(self.drop(F.elu(self.p6(pz))))))
        
        pzpred = self.p8(pz)
        
        return pzpred

    def predloss(self,y,p):
        return torch.mean(wDist(y,p)) #torch.mean(psig*wDist(y,p)) - 0.5*torch.mean(torch.log(psig)), 

# Test the predictor on a hold-out set to check for overfitting
def testPredictData(states,acts,ends):
    BS = 200
    predict.eval()
    err = 0
    count = 0
    idx = np.random.permutation(states.shape[0])
    
    for i in range(states.shape[0]//BS):
        s = tovar(states[idx[i*BS:i*BS+BS]])
        a = tovar(acts[idx[i*BS:i*BS+BS]])
        e = tovar(ends[idx[i*BS:i*BS+BS]])

        preds = predict.predict(s.unsqueeze(2),a.unsqueeze(2)).squeeze(2)
        loss = predict.predloss(e, preds) 
        
        err = err + loss.cpu().data.numpy()[0] 
        count += 1
    
    return err/float(count)

# Train the predictor for one epoch on the dataset 
def trainPredictData(states,acts,ends):
    BS = 50
    predict.train()
    err = 0
    count = 0
    idx = np.random.permutation(states.shape[0])
    
    for i in range(states.shape[0]//BS):
        s = tovar(states[idx[i*BS:i*BS+BS]])
        a = tovar(acts[idx[i*BS:i*BS+BS]])
        e = tovar(ends[idx[i*BS:i*BS+BS]])

        predict.zero_grad()
        preds = predict.predict(s.unsqueeze(2),a.unsqueeze(2)).squeeze(2)
        loss = predict.predloss(e, preds) 
        
        loss.backward()
        predict.adam.step()
        
        err = err + loss.cpu().data.numpy()[0] 
        count += 1
    
    return err/float(count)

### Proposer network

We borrow the first layer from the predictor to help the proposer interpret the environment. That layer is held constant while training the proposer, so it's just a kind of learned pre-processing.

In [ ]:
class Propose(nn.Module):
    def __init__(self):
        super(Propose,self).__init__()
        
        self.d1 = nn.Conv1d(2+HIDDEN, HIDDEN2, 1).cuda()
        self.d2 = nn.Conv1d(HIDDEN2, HIDDEN2, 1).cuda()
        self.d3 = nn.Conv1d(HIDDEN2, HIDDEN2, 1).cuda()
        self.d4 = nn.Conv1d(HIDDEN2, HIDDEN2, 1).cuda()
        self.d5 = nn.Conv1d(HIDDEN2, HIDDEN2, 1).cuda()
        self.d6 = nn.Conv1d(HIDDEN2, HIDDEN2, 1).cuda()        
        self.dprop = nn.Conv1d(HIDDEN2, ACTUATORS, 1).cuda()
        
        self.adam = torch.optim.Adam(self.parameters(), lr = 1e-5)
            
    def forward(self,idx):                
        z = F.elu(self.d1(idx))
        z = z+F.elu(self.d3(F.elu(self.d2(z))))
        z = z+F.elu(self.d5(F.elu(self.d4(z))))
        z = F.elu(self.d6(z))
        
        a = F.tanh(self.dprop(z))
        return a
    
    def generate(self, state, pred):
        pred.eval()
        BS = state.size()[0]
        xx,yy = np.meshgrid(np.arange(MESH)/float(MESH), \
                            np.arange(MESH)/float(MESH))
        xx = xx.ravel()
        yy = yy.ravel()
        
        idx_x = tovar(2.0*xx - 1.0).unsqueeze(0).unsqueeze(1).expand(BS,1,MESH*MESH)
        idx_y = tovar(2.0*yy - 1.0).unsqueeze(0).unsqueeze(1).expand(BS,1,MESH*MESH)
                
        state = state.unsqueeze(2)
        
        # This uses the predictor network's first layer to process the sensory input
        z = pred.process(state).expand(BS,HIDDEN,MESH*MESH)
        
        a = self.forward(torch.cat([z,idx_x,idx_y],1))
        
        ashp = a.size()
        
        a = a.view(ashp[0], ACTUATORS, MESH*MESH)
        x = pred.predict_hid(z,a)
        
        return a,x

def getProposals(states):
    acts, preds = propose.generate(tovar(states), predict)    
    
    return acts, preds

def measureMinDist(preds):
    BS = preds.size()[0]
    PROPOSALS = MESH*MESH
    # Now train the proposal network on distance between generated outcomes
    p1 = preds.unsqueeze(2).expand(BS,OSENSORS,PROPOSALS,PROPOSALS)
    p2 = preds.unsqueeze(3).expand(BS,OSENSORS,PROPOSALS,PROPOSALS)
    dp = torch.sum( wDist4(p1,p2), 1).squeeze(1)
    eye = tovar(np.eye(PROPOSALS)).unsqueeze(0).expand_as(dp)
    
    dp = dp + 1e6*eye
    mind,minidx = torch.min(dp,2)
    mind,minidx = torch.min(mind,1)
    loss2 = torch.mean(mind)
    
    return loss2

def evalProposals(acts, olist):
    outs = []
    pool = Pool(4)
    
    for j in range(acts.shape[2]):
        o = pool.map(simulate, zip([acts[i,:,j] for i in range(BS)], olist))
        outs.append(o)
    pool.close()
    pool.join()
    outs = np.array(outs)
    
    return outs

# Train the proposer for some number of epochs
def trainPropose(epochs):
    global MESH
    
    for epoch in range(epochs):
        idx = np.random.permutation(states.shape[0])

        err2 = 0
        count = 0
        
        for i in range(states.shape[0]//BS):
            propose.zero_grad()

            pacts, preds = getProposals(states[idx[i*BS:i*BS+BS]])

            pgrid = preds.view((BS,OSENSORS,MESH,MESH))
            delta = torch.mean((pgrid[:,:,:-1,:] - pgrid[:,:,1:,:])**2) + torch.mean((pgrid[:,:,:,:-1] - pgrid[:,:,:,1:])**2)
            loss2 = measureMinDist(preds)
            loss = -loss2 + 0.1*delta
            err2 += loss2.cpu().data.numpy()[0]
            count += 1

            loss.backward()

            propose.adam.step()
        
        # Log some training data
        f = open("mindist_propose.txt","a")
        f.write("%.6g\n" % (err2/float(count)))
        f.close()

        nm = MESH
        MESH = 17
        pacts, preds = getProposals(states[0:1])
        preds = preds.cpu().data.numpy()[0]
        np.savetxt("preds.txt", preds[7*3:7*3+3,:].T)

        MESH = nm
        
        # Save a checkpoint
        torch.save(propose.state_dict(), open("propose.th","wb"))

In [ ]:
# Instantiate the networks

predict = Predict()
propose = Propose()

# To load from checkpoint, uncomment this
#predict.load_state_dict(torch.load("predict.th"))
#propose.load_state_dict(torch.load("propose.th"))

### Collecting data

In [ ]:
states = np.zeros((0,SENSORS))
acts = np.zeros((0,ACTUATORS))
ends = np.zeros((0,OSENSORS))

# If you've already generated the data, uncomment to load it in

states = np.load("states.npy")
acts = np.load("acts.npy")
ends = np.load("ends.npy")

In [ ]:
# Generate the data in chunks
# Makes it easy to add variable amounts onto the existing dataset

# This one generates random action playouts
for i in range(1):
    s, a, e = getData(N=500)
    states = np.vstack([states,s])
    acts = np.vstack([acts,a])
    ends = np.vstack([ends,e])
    
np.save("states.npy",states)
np.save("acts.npy",acts)
np.save("ends.npy",ends)

In [ ]:
# This one generates playouts based on the current proposal network
for i in range(1):
    s, a, e = getProposeData(N=500)
    states = np.vstack([states,s])
    acts = np.vstack([acts,a])
    ends = np.vstack([ends,e])
    
np.save("states.npy",states)
np.save("acts.npy",acts)
np.save("ends.npy",ends)

### Train predictor

In [ ]:
# Train/test division
idx = np.random.permutation(states.shape[0])

test_states = states[idx[0:5000]]
test_acts = acts[idx[0:5000]]
test_ends = ends[idx[0:5000]]

states = states[idx[5000:]]
acts = acts[idx[5000:]]
ends = ends[idx[5000:]]

In [ ]:
# Train the predictor for 200 epochs

for epoch in range(200):
    lr = 2.5e-4
    if epoch>100:
        lr = 1e-4

    for param_group in predict.adam.param_groups:
        param_group['lr'] = lr

    err = trainPredictData(states,acts,ends)
    terr = testPredictData(test_states,test_acts,test_ends)
    
    # Log the training progress
    f = open("error.txt","a")
    f.write("%d %.6g %.6g\n" % (epoch,err,terr))
    f.close()

    # Save the model as we train
    torch.save(predict.state_dict(), open("predict.th","wb"))

### Train proposer

In [ ]:
# Train the proposer for 100 epochs

BS = 100
MESH = 9

trainPropose(100)